In [ ]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [3]:
import os
if not os.path.isdir('saved_models_mnist_{}'.format('gan')):
    os.mkdir('saved_models_mnist_{}'.format('gan'))
f = open('saved_models_mnist_{}/log_collapse1.txt'.format('gan'), mode='w')
import os
import tensorflow as tf

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam #optimizer of keras

import matplotlib.pyplot as plt

import sys

import numpy as np
import os
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels) #shape of image
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5) #optimizer of gan

        # Build and compile the discriminator,only to keras
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))  #Input():用来实例化一个keras张量
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (X_test, y_test) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.

        X_train = np.expand_dims(X_train, axis=3)  #expand_dims用于扩充数组形状
        print(np.shape(X_train))
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_test = X_test / 127.5 - 1.
        X_test = np.expand_dims(X_test, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        global_step=0
        nb_batches = int(X_train.shape[0] / batch_size)

        for epoch in range(epochs):
            for index in range(nb_batches):
                global_step += 1
                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Select a random batch of images
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                imgs = X_train[idx]

                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Train the generator (to have the discriminator label samples as valid)
                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss))

                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = sample_num//10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
#         sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')

if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30, batch_size=64, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

0 [D loss: 0.031504, acc.: 100.00%] [G loss: 4.426101]
0 [D loss: 0.105991, acc.: 96.88%] [G loss: 4.243402]
0 [D loss: 0.034515, acc.: 100.00%] [G loss: 4.467851]
0 [D loss: 0.156593, acc.: 92.19%] [G loss: 3.934579]
0 [D loss: 0.044805, acc.: 99.22%] [G loss: 4.452204]
0 [D loss: 0.180746, acc.: 91.41%] [G loss: 4.227439]
0 [D loss: 0.035462, acc.: 100.00%] [G loss: 4.297255]
0 [D loss: 0.235736, acc.: 91.41%] [G loss: 3.610128]
0 [D loss: 0.072945, acc.: 98.44%] [G loss: 4.193753]
0 [D loss: 0.235207, acc.: 89.84%] [G loss: 3.570673]
0 [D loss: 0.085355, acc.: 97.66%] [G loss: 4.412085]
0 [D loss: 1.145715, acc.: 62.50%] [G loss: 3.462661]
0 [D loss: 0.669702, acc.: 73.44%] [G loss: 2.583945]
0 [D loss: 0.136529, acc.: 95.31%] [G loss: 3.350923]
0 [D loss: 0.075894, acc.: 98.44%] [G loss: 3.767818]
0 [D loss: 0.056677, acc.: 98.44%] [G loss: 3.842334]
0 [D loss: 0.077568, acc.: 97.66%] [G loss: 3.791775]
0 [D loss: 0.086685, acc.: 99.22%] [G loss: 3.568545]
0 [D loss: 0.053455, acc.

0 [D loss: 0.730852, acc.: 46.09%] [G loss: 1.253151]
0 [D loss: 0.711416, acc.: 48.44%] [G loss: 1.187093]
0 [D loss: 0.756355, acc.: 41.41%] [G loss: 1.099946]
0 [D loss: 0.722905, acc.: 46.09%] [G loss: 1.041457]
0 [D loss: 0.746457, acc.: 46.88%] [G loss: 1.029425]
0 [D loss: 0.800937, acc.: 38.28%] [G loss: 0.929626]
0 [D loss: 0.680763, acc.: 48.44%] [G loss: 1.077572]
0 [D loss: 0.725002, acc.: 53.12%] [G loss: 0.925382]
0 [D loss: 0.822230, acc.: 41.41%] [G loss: 0.813829]
0 [D loss: 0.712595, acc.: 46.09%] [G loss: 0.988501]
0 [D loss: 0.765912, acc.: 40.62%] [G loss: 0.899539]
0 [D loss: 0.728069, acc.: 46.09%] [G loss: 0.817260]
0 [D loss: 0.806347, acc.: 42.19%] [G loss: 0.839034]
0 [D loss: 0.693424, acc.: 46.88%] [G loss: 0.976470]
0 [D loss: 0.752955, acc.: 43.75%] [G loss: 0.815687]
0 [D loss: 0.740903, acc.: 42.97%] [G loss: 0.787396]
0 [D loss: 0.831852, acc.: 35.16%] [G loss: 0.668880]
0 [D loss: 0.731953, acc.: 44.53%] [G loss: 0.763885]
0 [D loss: 0.757184, acc.: 4

##############
[ 6.8807581   7.59974359 10.54166883  8.72363101  8.63168545 10.98333983
 11.27914699  8.75477063  8.87364841  8.04824191]
##########
0 [D loss: 0.636095, acc.: 50.78%] [G loss: 0.689271]
0 [D loss: 0.648942, acc.: 49.22%] [G loss: 0.695028]
0 [D loss: 0.647855, acc.: 50.00%] [G loss: 0.679717]
0 [D loss: 0.653098, acc.: 50.00%] [G loss: 0.695227]
0 [D loss: 0.650435, acc.: 50.78%] [G loss: 0.702664]
0 [D loss: 0.646571, acc.: 53.12%] [G loss: 0.684976]
0 [D loss: 0.650469, acc.: 51.56%] [G loss: 0.678707]
0 [D loss: 0.659057, acc.: 50.78%] [G loss: 0.680736]
0 [D loss: 0.664791, acc.: 50.78%] [G loss: 0.677064]
0 [D loss: 0.675759, acc.: 49.22%] [G loss: 0.674505]
0 [D loss: 0.685007, acc.: 45.31%] [G loss: 0.666304]
0 [D loss: 0.672700, acc.: 49.22%] [G loss: 0.678373]
0 [D loss: 0.657875, acc.: 50.00%] [G loss: 0.703097]
0 [D loss: 0.655472, acc.: 47.66%] [G loss: 0.713294]
0 [D loss: 0.670463, acc.: 47.66%] [G loss: 0.712935]
0 [D loss: 0.650169, acc.: 50.78%] [G los

0 [D loss: 0.624439, acc.: 60.94%] [G loss: 0.777937]
0 [D loss: 0.635282, acc.: 59.38%] [G loss: 0.764908]
0 [D loss: 0.652044, acc.: 52.34%] [G loss: 0.730976]
0 [D loss: 0.654702, acc.: 56.25%] [G loss: 0.728111]
0 [D loss: 0.638478, acc.: 53.91%] [G loss: 0.743117]
0 [D loss: 0.653724, acc.: 55.47%] [G loss: 0.752222]
0 [D loss: 0.659556, acc.: 57.81%] [G loss: 0.747867]
0 [D loss: 0.637101, acc.: 59.38%] [G loss: 0.746462]
0 [D loss: 0.655508, acc.: 56.25%] [G loss: 0.740498]
0 [D loss: 0.670246, acc.: 48.44%] [G loss: 0.746007]
0 [D loss: 0.666396, acc.: 47.66%] [G loss: 0.737506]
0 [D loss: 0.661090, acc.: 50.00%] [G loss: 0.727316]
0 [D loss: 0.667382, acc.: 50.00%] [G loss: 0.731132]
0 [D loss: 0.643175, acc.: 58.59%] [G loss: 0.739860]
0 [D loss: 0.682200, acc.: 51.56%] [G loss: 0.723878]
0 [D loss: 0.689214, acc.: 47.66%] [G loss: 0.705860]
0 [D loss: 0.676904, acc.: 50.78%] [G loss: 0.723637]
0 [D loss: 0.655994, acc.: 60.94%] [G loss: 0.735185]
0 [D loss: 0.652470, acc.: 5

...

29 [D loss: 0.660403, acc.: 57.81%] [G loss: 0.875453]
29 [D loss: 0.657212, acc.: 56.25%] [G loss: 0.924472]
29 [D loss: 0.647642, acc.: 61.72%] [G loss: 0.956464]
29 [D loss: 0.627011, acc.: 62.50%] [G loss: 0.892769]
29 [D loss: 0.682372, acc.: 55.47%] [G loss: 0.917659]
29 [D loss: 0.647347, acc.: 62.50%] [G loss: 0.945801]
29 [D loss: 0.662282, acc.: 59.38%] [G loss: 0.926595]
29 [D loss: 0.640800, acc.: 62.50%] [G loss: 1.011948]
29 [D loss: 0.624381, acc.: 64.06%] [G loss: 0.944198]
29 [D loss: 0.668258, acc.: 58.59%] [G loss: 0.974086]
29 [D loss: 0.635339, acc.: 64.84%] [G loss: 0.917414]
29 [D loss: 0.598393, acc.: 69.53%] [G loss: 0.947416]
29 [D loss: 0.719847, acc.: 51.56%] [G loss: 0.941038]
29 [D loss: 0.599501, acc.: 70.31%] [G loss: 0.897489]
29 [D loss: 0.623763, acc.: 64.06%] [G loss: 0.917056]
29 [D loss: 0.672626, acc.: 65.62%] [G loss: 0.979906]
29 [D loss: 0.623789, acc.: 67.19%] [G loss: 1.025561]
29 [D loss: 0.636462, acc.: 63.28%] [G loss: 1.008116]
29 [D loss

29 [D loss: 0.649673, acc.: 64.06%] [G loss: 0.896892]
29 [D loss: 0.641088, acc.: 60.94%] [G loss: 0.861158]
29 [D loss: 0.602556, acc.: 64.84%] [G loss: 0.960052]
29 [D loss: 0.648133, acc.: 60.94%] [G loss: 0.888195]
29 [D loss: 0.622916, acc.: 61.72%] [G loss: 0.930731]
29 [D loss: 0.651877, acc.: 61.72%] [G loss: 1.023320]
29 [D loss: 0.610173, acc.: 71.09%] [G loss: 0.902117]
29 [D loss: 0.708073, acc.: 49.22%] [G loss: 0.890355]
29 [D loss: 0.696391, acc.: 53.12%] [G loss: 0.912116]
29 [D loss: 0.640662, acc.: 65.62%] [G loss: 0.906424]
29 [D loss: 0.722308, acc.: 56.25%] [G loss: 0.941781]
29 [D loss: 0.631631, acc.: 64.06%] [G loss: 0.925654]
29 [D loss: 0.618014, acc.: 67.97%] [G loss: 0.883161]
29 [D loss: 0.625129, acc.: 62.50%] [G loss: 1.033914]
29 [D loss: 0.640789, acc.: 62.50%] [G loss: 0.960758]
29 [D loss: 0.691062, acc.: 53.12%] [G loss: 0.958127]
29 [D loss: 0.671036, acc.: 59.38%] [G loss: 0.920545]
29 [D loss: 0.661144, acc.: 56.25%] [G loss: 0.902978]
29 [D loss

29 [D loss: 0.660352, acc.: 64.06%] [G loss: 0.936506]
29 [D loss: 0.682444, acc.: 53.12%] [G loss: 0.886484]
29 [D loss: 0.632595, acc.: 64.84%] [G loss: 0.961970]
29 [D loss: 0.663053, acc.: 61.72%] [G loss: 0.946190]
29 [D loss: 0.652684, acc.: 64.06%] [G loss: 0.880044]
29 [D loss: 0.601504, acc.: 67.19%] [G loss: 0.867860]
29 [D loss: 0.649311, acc.: 58.59%] [G loss: 0.885106]
29 [D loss: 0.643101, acc.: 59.38%] [G loss: 0.843644]
29 [D loss: 0.676367, acc.: 58.59%] [G loss: 0.933573]
29 [D loss: 0.670226, acc.: 59.38%] [G loss: 0.962642]
29 [D loss: 0.626601, acc.: 67.19%] [G loss: 0.908171]
29 [D loss: 0.671278, acc.: 58.59%] [G loss: 0.914214]
29 [D loss: 0.638423, acc.: 68.75%] [G loss: 0.943880]
##############
[2.78852068 2.00187588 6.02760305 4.518771   3.6845442  5.29051549
 4.65353092 5.08294018 4.42782923 4.09544104]
##########
29 [D loss: 0.704945, acc.: 54.69%] [G loss: 0.920630]
29 [D loss: 0.608320, acc.: 68.75%] [G loss: 0.903711]
29 [D loss: 0.649368, acc.: 62.50%] 

29 [D loss: 0.629834, acc.: 63.28%] [G loss: 0.909734]
29 [D loss: 0.586419, acc.: 71.88%] [G loss: 0.956816]
29 [D loss: 0.710219, acc.: 57.03%] [G loss: 0.866735]
29 [D loss: 0.623004, acc.: 65.62%] [G loss: 0.900215]
29 [D loss: 0.645274, acc.: 61.72%] [G loss: 0.866341]
29 [D loss: 0.661496, acc.: 57.81%] [G loss: 0.913857]
29 [D loss: 0.654046, acc.: 60.94%] [G loss: 0.992676]
29 [D loss: 0.665192, acc.: 56.25%] [G loss: 0.903869]
29 [D loss: 0.659302, acc.: 60.94%] [G loss: 0.971044]
29 [D loss: 0.662192, acc.: 55.47%] [G loss: 0.929196]
29 [D loss: 0.679616, acc.: 55.47%] [G loss: 0.874922]
29 [D loss: 0.650352, acc.: 60.94%] [G loss: 0.843758]
29 [D loss: 0.646787, acc.: 60.16%] [G loss: 0.852872]
29 [D loss: 0.673226, acc.: 60.94%] [G loss: 0.949271]
29 [D loss: 0.625110, acc.: 65.62%] [G loss: 0.922876]
29 [D loss: 0.657342, acc.: 64.84%] [G loss: 0.892246]
29 [D loss: 0.673524, acc.: 60.94%] [G loss: 0.903576]
29 [D loss: 0.681757, acc.: 54.69%] [G loss: 0.924898]
29 [D loss

29 [D loss: 0.639922, acc.: 60.16%] [G loss: 0.900180]
29 [D loss: 0.661687, acc.: 57.81%] [G loss: 0.888691]
29 [D loss: 0.643029, acc.: 62.50%] [G loss: 0.851108]
29 [D loss: 0.668996, acc.: 55.47%] [G loss: 0.964681]
29 [D loss: 0.675178, acc.: 57.03%] [G loss: 0.934810]
29 [D loss: 0.634891, acc.: 64.84%] [G loss: 0.986056]
29 [D loss: 0.662124, acc.: 56.25%] [G loss: 0.887107]
29 [D loss: 0.630641, acc.: 66.41%] [G loss: 0.896385]
29 [D loss: 0.659914, acc.: 61.72%] [G loss: 0.837907]
29 [D loss: 0.653236, acc.: 64.84%] [G loss: 0.876173]
29 [D loss: 0.665917, acc.: 53.12%] [G loss: 0.949852]
29 [D loss: 0.630558, acc.: 64.06%] [G loss: 1.032453]
29 [D loss: 0.614884, acc.: 67.97%] [G loss: 0.920106]
29 [D loss: 0.660681, acc.: 60.94%] [G loss: 0.998246]
29 [D loss: 0.655519, acc.: 62.50%] [G loss: 0.963947]
29 [D loss: 0.644144, acc.: 60.16%] [G loss: 0.904363]
29 [D loss: 0.630918, acc.: 60.16%] [G loss: 0.944118]
29 [D loss: 0.605724, acc.: 64.06%] [G loss: 0.919592]
29 [D loss